In [1]:
import pandas as pd
import ast
import json

%load_ext autoreload
%autoreload 2
import utils as ut

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Ruta al dataset australian_user_reviews
review_data = 'data/json/australian_user_reviews.json'

rows = []
with open(review_data, encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

# Converts to dataframe
df_reviews = pd.DataFrame(rows)
df_reviews


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
ut.data_summ(df_reviews)


Total rows:  25799

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,user_id,[<class 'str'>],25799,0.0,0
1,user_url,[<class 'str'>],25799,0.0,0
2,reviews,[<class 'list'>],25799,0.0,0


We going to analize duplicate rows by user

In [4]:
duplicated = ut.duplicates(df_reviews, 'user_id')
duplicated

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


In [5]:
df_reviews = ut.drop_duplicates(df_reviews, 'user_id')

In [6]:
df_reviews.shape

(25485, 3)

Now, We are going to expand the data that is nested in the reviews column

In [7]:
unpacked_data = []
for index, row in df_reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    for review_dict in row['reviews']:
        review_dict['user_id'] = user_id
        review_dict['user_url'] = user_url
        unpacked_data.append(review_dict)

df_reviews = pd.DataFrame(unpacked_data)

In [8]:
ut.data_summ(df_reviews)


Total rows:  58430

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,funny,[<class 'str'>],58430,0.0,0
1,posted,[<class 'str'>],58430,0.0,0
2,last_edited,[<class 'str'>],58430,0.0,0
3,item_id,[<class 'str'>],58430,0.0,0
4,helpful,[<class 'str'>],58430,0.0,0
5,recommend,[<class 'bool'>],58430,0.0,0
6,review,[<class 'str'>],58430,0.0,0
7,user_id,[<class 'str'>],58430,0.0,0
8,user_url,[<class 'str'>],58430,0.0,0


In [9]:
df_reviews[df_reviews['user_id']=='maplemage']

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
18,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud,maplemage,http://steamcommunity.com/id/maplemage
19,1 person found this review funny,"Posted December 23, 2013.",,211820,12 of 16 people (75%) found this review helpful,True,"It's like Terraria, you play for 9 hours strai...",maplemage,http://steamcommunity.com/id/maplemage
20,2 people found this review funny,"Posted March 14, 2014.",,730,5 of 5 people (100%) found this review helpful,True,"Hold shift to win, Hold CTRL to lose.",maplemage,http://steamcommunity.com/id/maplemage
21,,"Posted July 11, 2013.",,204300,No ratings yet,True,"OH YES, THIS GAME IS THE BEST, THEY ADD STUFF ...",maplemage,http://steamcommunity.com/id/maplemage


In [10]:
df_reviews['funny'][0]

''

In [11]:
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,None,"Posted July 15, 2011.",None,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,None,"Posted April 21, 2011.",None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,None,"Posted September 8, 2013.",None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637


In [12]:
ut.data_summ(df_reviews)


Total rows:  58430

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,funny,"[<class 'NoneType'>, <class 'str'>]",8010,86.29,50420
1,posted,[<class 'str'>],58430,0.00,0
2,last_edited,"[<class 'NoneType'>, <class 'str'>]",6037,89.67,52393
3,item_id,[<class 'str'>],58430,0.00,0
4,helpful,[<class 'str'>],58430,0.00,0
5,recommend,[<class 'bool'>],58430,0.00,0
6,review,"[<class 'str'>, <class 'NoneType'>]",58400,0.05,30
7,user_id,[<class 'str'>],58430,0.00,0
8,user_url,[<class 'str'>],58430,0.00,0


We replace null data in the dataframe

In [13]:
ut.replace_all_nulls(df_reviews)

In [14]:
df_reviews

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,No data,"Posted November 5, 2011.",No data,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,No data,"Posted July 15, 2011.",No data,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,No data,"Posted April 21, 2011.",No data,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,No data,"Posted June 24, 2014.",No data,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,No data,"Posted September 8, 2013.",No data,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
58425,No data,Posted July 10.,No data,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58426,No data,Posted July 8.,No data,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58427,1 person found this review funny,Posted July 3.,No data,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
58428,No data,Posted July 20.,No data,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In order to clean the dataframe focused in our objective, We going to extarct year in column 'posted'

In [15]:
import re

def date_post(date_str):
    date_r = re.search(r'(\w+\s\d{1,2},\s\d{4})', date_str)
    if date_r:
        date_rvw = date_r.group(1)
        try:
            date_rt = pd.to_datetime(date_rvw)
            return date_rt.strftime('%Y')
        except ValueError:
            return 'Invalid Date'
    else:
        return 'Invalid Format'

df_reviews['posted'] = df_reviews['posted'].apply(date_post)


In [16]:
df_reviews = df_reviews[df_reviews['posted'] != 'Invalid Format']
df_reviews['posted'] 

0        2011
1        2011
2        2011
3        2014
4        2013
         ... 
58377    2015
58380    2015
58390    2015
58392    2015
58401    2015
Name: posted, Length: 48498, dtype: object

Now, we going to drop clumns no relevant for our project

In [17]:
df_reviews = df_reviews.drop(["funny", "last_edited", "helpful", "user_url"], axis=1)

In [18]:
ut.data_summ(df_reviews)


Total rows:  48498

Total full null rows:  0


,Column,Data_type,No_miss_Qty,%Missing,Missing_Qty
0,posted,[<class 'str'>],48498,0.0,0
1,item_id,[<class 'str'>],48498,0.0,0
2,recommend,[<class 'bool'>],48498,0.0,0
3,review,[<class 'str'>],48498,0.0,0
4,user_id,[<class 'str'>],48498,0.0,0


In [19]:
dataframe = [df_reviews]
name = ['ureviews_clean']
ut.save_to_csv(dataframe, name)

DataFrame 'ureviews_clean' saved as 'data/csv/ureviews_clean.csv'
